In [1]:
import pandas as pd

df_texts = pd.read_csv('../data/proionta_tis_fisis_votana_annotated.csv', index_col=None, converters={'entities': eval})

In [3]:
df_texts['text_entities'] = list(zip(df_texts.text, df_texts.entities))

In [5]:
DATA = [(text, {'entities': entities}) for (text, entities) in df_texts['text_entities']]

In [7]:
from sklearn.model_selection import train_test_split
TRAIN_DATA, TEST_DATA = train_test_split(DATA, test_size=0.25, random_state=10)

In [8]:
print(len(DATA))
print(len(TRAIN_DATA))
print(len(TEST_DATA))

668
501
167


In [10]:
from __future__ import unicode_literals, print_function

import plac
import random
import warnings
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding


# training data
# Note: If you're using an existing model, make sure to mix in examples of
# other entity types that spaCy correctly recognized before. Otherwise, your
# model might learn the new type, but "forget" what it previously knew.
# https://explosion.ai/blog/pseudo-rehearsal-catastrophic-forgetting


def main(model=None, new_model_name="ethnopharmacology", output_dir=None, n_iter=30):
    """Set up the pipeline and entity recognizer, and train the new entity."""
    random.seed(0)
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("el")  # create blank Language class
        print("Created blank 'el' model")
    # Add entity recognizer to model if it's not in the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner)
    # otherwise, get it, so we can add labels to it
    else:
        ner = nlp.get_pipe("ner")

    ner.add_label("ORGAN")  # add new entity label to entity recognizer
    # Adding extraneous labels shouldn't mess anything up
    ner.add_label("PLANT")
    if model is None:
        optimizer = nlp.begin_training()
    else:
        optimizer = nlp.resume_training()
    move_names = list(ner.move_names)
    # get names of other pipes to disable them during training
    pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
    # only train NER
    with nlp.disable_pipes(*other_pipes), warnings.catch_warnings():
        # show warnings for misaligned entity spans once
        warnings.filterwarnings("once", category=UserWarning, module='spacy')

        sizes = compounding(1.0, 4.0, 1.001)
        # batch up the examples using spaCy's minibatch
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            batches = minibatch(TRAIN_DATA, size=sizes)
            losses = {}
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
            print("Losses", losses)

    # test the trained model
    test_texts = [a for (a, b) in TEST_DATA]
    for test_text in test_texts:
        doc = nlp(test_text)
        print("Entities in '%s'" % test_text[:50])
        for ent in doc.ents:
            print(ent.label_, ent.text)

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.meta["name"] = new_model_name  # rename model
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        # Check the classes have loaded back consistently
        assert nlp2.get_pipe("ner").move_names == move_names
        doc2 = nlp2(test_text)
        for ent in doc2.ents:
            print(ent.label_, ent.text)



In [11]:
main(n_iter=50)

Created blank 'el' model
Losses {'ner': 35829.3873170098}
Losses {'ner': 12842.634617271493}
Losses {'ner': 7842.006587967364}
Losses {'ner': 5564.386972023692}
Losses {'ner': 4214.817096372475}
Losses {'ner': 3504.5040673387825}
Losses {'ner': 2889.135276354979}
Losses {'ner': 2464.6884763355783}
Losses {'ner': 2266.6167769538124}
Losses {'ner': 2129.1352449631}
Losses {'ner': 1820.6282447643998}
Losses {'ner': 1628.1213614037054}
Losses {'ner': 1642.4387587244692}
Losses {'ner': 1409.6656313668714}
Losses {'ner': 1346.150831042147}
Losses {'ner': 1246.2702931791553}
Losses {'ner': 1321.1300723513186}
Losses {'ner': 1176.4790158372593}
Losses {'ner': 1185.1244782494011}
Losses {'ner': 1039.0307823781707}
Losses {'ner': 949.0520046937113}
Losses {'ner': 933.0336257745201}
Losses {'ner': 896.8046860418525}
Losses {'ner': 824.797211482398}
Losses {'ner': 824.8012421910067}
Losses {'ner': 727.3938787109452}
Losses {'ner': 718.7837936396401}
Losses {'ner': 754.0214610653604}
Losses {'ner':

Entities in '  Βότανα  Σμύρνα, η ιερή!ΒόταναΣμύρνα, η ιερή!από '
PLANT Σμύρνα
PLANT σμύρνα
PLANT μύρρο
PLANT λιβάνι
PLANT λιβανιού
PLANT μύρρο
PLANT ψύλλους
PLANT μύρρου
PLANT σμύρνα
PLANT σμύρνα
ORGAN γλώσσα
ORGAN βλεννογόνων
ORGAN λάρυγγα
PLANT σμύρνα
PLANT μύρρου
PLANT Μύρρο
Entities in '  Βότανα  Gotu Kola-Σεντέλα, ενεργοποιεί νευρικό σ'
ORGAN νευρικό
ORGAN εγκέφαλο
ORGAN νευρικό
ORGAN εγκέφαλο
ORGAN νευρικό
ORGAN εγκέφαλο
PLANT Γκότου
PLANT γιόγκι
ORGAN εγκεφάλου
ORGAN νευρικού
ORGAN εγκέφαλο
ORGAN φλεβική
ORGAN νευρικό
ORGAN νευρικού
ORGAN νευρικές
ORGAN νευρικό
PLANT λαχανικά
ORGAN εγκεφαλικό
ORGAN εγκεφάλου
ORGAN φλέβες
ORGAN φλεβικά
ORGAN νευρικό
ORGAN νευρικό
ORGAN δέρματος
ORGAN επινεφριδίων
ORGAN φλεβών
ORGAN τριχοειδών αγγείων
ORGAN φλεβικής
ORGAN φλέβες
ORGAN αιμοφόρων αγγείων
ORGAN μάτια
ORGAN δέρμα
PLANT Κισσού
ORGAN πνεύμονες
ORGAN ηπατική
ORGAN στομάχου
ORGAN φλεβικής
PLANT κρεμμύδια
PLANT λεμονιού
PLANT τσίλι
PLANT καρύδας
ORGAN μυαλό
PLANT πιπεριού
PLANT γάλα
PLANT 

Entities in '  Βότανα  Αντιγριπικά-Αντιιικά βότανα και τροφέςΒό'
PLANT Εχινάκεια
PLANT Εχινάκεια
PLANT μέντας
PLANT Μέντα
PLANT Θυμάρι
PLANT θυμαριού
PLANT θυμάρι
PLANT Θυμάρι
PLANT Τσουκνίδα
PLANT τσουκνίδα
PLANT Τσουκνίδα
PLANT Καλεντούλα
PLANT καλεντούλας
ORGAN δέρματος
PLANT Καλέντουλα
ORGAN Αιμοφόρα Αγγεία
PLANT Δίκταμος
PLANT Δίκταμο
PLANT Λεβάντα
PLANT λεβάντα
ORGAN εντέρου
PLANT λεβάντα
PLANT Λεβάντα
PLANT Τίλιο
ORGAN νεύρα
PLANT Τίλιο
PLANT Φλαμούρι
PLANT Ευκάλυπτος
PLANT ευκαλύπτου
PLANT λεμόνι
PLANT ευκαλύπτου
PLANT Ευκάλυπτος
PLANT Χαμομήλι
PLANT χαμομηλιού
PLANT Χαμομήλι
PLANT χαμομήλι
PLANT μαύρο τσάι
PLANT τσάι του βουνού
PLANT Τσάι του βουνού
PLANT μολόχα
PLANT Αγριομολόχα
PLANT Μελισσόχορτο
PLANT βάλσαμο
PLANT μελισσόχορτο
PLANT Μελισσόχορτο
ORGAN νευρικό
PLANT Φασκόμηλο
PLANT φασκόμηλου
PLANT Φασκόμηλο
PLANT Τζίντζερ
PLANT τζίντζερ
PLANT Φλαμουριά
PLANT Τίλιο
PLANT Φλαμούρι
ORGAN πνευμονία
PLANT Γλυκόριζα
PLANT Σκόρδο
PLANT Σκόρδο
PLANT Κρεμμύδι
PLANT κρεμμύδι
PLANT κ

Entities in '  Βότανα  Η μαγεία των φυτικών βαμμάτων και ο ρόλο'
PLANT Τζίνσενγκ
PLANT τζίνσενγκ
ORGAN στομαχικές
PLANT Τζίνσενγκ
PLANT Ginseng
PLANT Κρόκος
PLANT τζίνσενγκ
PLANT Ίνουλα
ORGAN πνεύμονες
PLANT Ινούλα
ORGAN πνευμόνων
PLANT καλέντουλα
PLANT καλέντουλας
PLANT χρυσάνθεμα
PLANT Καλέντουλα
PLANT καλέντουλας
PLANT Κανέλα
PLANT κανέλας
PLANT κανέλα
PLANT κανέλας
PLANT κανέλα
ORGAN στομάχι
PLANT κανέλας
PLANT κανέλα
PLANT κανέλα
PLANT κανέλας
ORGAN εγκεφαλικά
PLANT κανέλας
ORGAN μήτρας
ORGAN δέρμα
PLANT κανέλα
PLANT Κανέλα
PLANT Κασσία
PLANT Κάρδαμο
ORGAN ήπαρ
ORGAN στομάχι
PLANT Κάρδαμο
PLANT Καρύδι
PLANT καρυδιών
PLANT καρύδια
PLANT καρύδια
PLANT καρύδια
ORGAN αιμοφόρα αγγεία
PLANT καρυδιού
ORGAN αιμοφόρων αγγείων
ORGAN αρτηρίες
ORGAN εγκεφαλικών
ORGAN οστικής
PLANT καρύδια
ORGAN εγκεφάλου
PLANT καρύδια
ORGAN αρτηριών
ORGAN αρτηριών
PLANT καρύδια
PLANT καρυδιών
PLANT καρύδια
ORGAN προστάτη
ORGAN μαστού
PLANT καρυδιών
ORGAN προστάτη
ORGAN μαστού
PLANT καρυδιών
PLANT καρύδια
ORGA

Entities in '  Βότανα  Αρετιμισία-Αψιθιά: Ισχυρό αντιμικροβιακό'
PLANT γλυκάνισο
PLANT μάραθο
ORGAN εντερικών
ORGAN στομάχι
ORGAN μαστού
ORGAN μαστού
ORGAN μαστού
ORGAN μαστού
PLANT Αρτεμισία
ORGAN εντερικών
ORGAN νηματώδεις
ORGAN έντερα
PLANT καρύδι
PLANT γαρύφαλλο
PLANT αψιθιά
PLANT αψίνθου
ORGAN πνευμονία
PLANT αψίνθου
ORGAN εντερικό
ORGAN δέρμα
ORGAN βλεννογόνους
PLANT αψιθιά
ORGAN εντέρων
PLANT αψιθιά
PLANT ρίγανη
PLANT θυμαριού
PLANT βερβερίνης
PLANT αψιθιά
ORGAN έντερο
PLANT αψιθιά
PLANT αψιθιάς
PLANT αψιθιά
PLANT φασκόμηλο
PLANT αψιθιάς
PLANT αψιθιά
PLANT αψιθιά
PLANT αψιθιά
ORGAN συριγγικό
PLANT αψιθιάς
PLANT αψιθιά
ORGAN εντερικό
ORGAN εντερικές
ORGAN στομάχου
ORGAN νευρικό
PLANT γλυκάνισου
PLANT αψιθιά
PLANT αψιθιάς
PLANT αψιθιάς
PLANT αψιθιά
PLANT αψιθιά
PLANT αψιθιά
PLANT αψιθιά
PLANT μέντα
PLANT γλυκάνισο
PLANT αψιθιά
ORGAN εντερικές
PLANT αψιθιάς
PLANT αψιθιά
PLANT αψιθιά
PLANT αψιθιά
PLANT αψιθιάς
Entities in '  Βότανα  «Φάρμακο» τα χόρτα της ελληνικής γης και'
PLANT χό

...

Entities in '  Βότανα  Η μαγεία των βαμμάτων: Ο ρόλος τους στην'
PLANT Βαλεριάνα
PLANT βαλεριάνα
PLANT Βαλεριάνα
PLANT σπαθόχορτο
PLANT σπαθόχορτο
PLANT βαλσαμόχορτο
ORGAN νεύρων
ORGAN νεύρων
ORGAN μαλλιών
ORGAN στομαχικές
ORGAN αρθρώσεων
ORGAN στομάχου
PLANT Βαλσαμόχορτο
PLANT βαλσαμόχορτου
PLANT Βασιλικός
ORGAN νευρικούς
ORGAN εντερικές
PLANT Βασιλικός
ORGAN αρθρώσεις
ORGAN πνευμόνων
PLANT Βιβούρνο
PLANT βιβούρνου
PLANT Πανσές
PLANT Βίσκος
PLANT Γκι
PLANT ιξός
PLANT Ιξός
PLANT ιξού
PLANT ιξού
PLANT ιξού
PLANT ιξός
ORGAN καρδιάς
ORGAN νευρικές
PLANT Βρώμη
ORGAN νευρικού
ORGAN οστά
ORGAN οστά
ORGAN καρδιά
PLANT βρώμη
PLANT βρώμης
ORGAN εντέρου
PLANT βρώμη
ORGAN στομάχι
PLANT βρώμη
PLANT βρώμη
PLANT βρώμη
PLANT βρώμης
PLANT σιτάρι
PLANT καλαμποκιού
PLANT σιτάρι
PLANT βρώμη
ORGAN εντέρου
PLANT βρώμη
PLANT βρώμη
ORGAN μαστού
ORGAN εντέρου
PLANT βρώμη
PLANT βρώμης
PLANT βρώμης
PLANT γάλα
PLANT βρώμης
PLANT βρώμη
PLANT βρώμης
PLANT βρώμης
PLANT Βρώμη
PLANT Βρώμης
PLANT βρώμη
Entities in '  

Entities in '  Βότανα  Matcha, ρόφημα με πολλαπλάσια θρεπτική α'
PLANT πράσινο τσάι
PLANT γάλα
ORGAN εντέρου
ORGAN μαστού
ORGAN ωοθηκών
ORGAN προστάτη
ORGAN πνεύμονα
PLANT πράσινο τσάι
PLANT πράσινο τσάι
PLANT πράσινο τσάι
PLANT ρόδι
ORGAN οστική
ORGAN αρτηριών
ORGAN συκωτιού
PLANT Πράσινο τσάι
Entities in '  Βότανα  Πικραγγουριά: Μαθαίνουμε και ενημερώνουμ'
PLANT Πικραγγουριά
PLANT Πικραγγουριά
PLANT πικραγγουριά
PLANT κολοκυθιά
PLANT αγγουριά
PLANT αγγούρια
ORGAN δέρμα
PLANT πικραγγουριά
PLANT γάλα
PLANT πικραγγουριά
ORGAN ματιών
ORGAN δοντιών
ORGAN αυτιών
ORGAN νεφρικά
PLANT πικραγγουριάς
ORGAN ήπατος
ORGAN βλεννογόνου
PLANT σταφυλής
ORGAN βλεννογόνου
ORGAN μύτης
ORGAN νεφρά
ORGAN αρθρώσεων
ORGAN οισοφάγου
ORGAN στομάχου
PLANT πικραγγουριάς
ORGAN βλεννογόνο
ORGAN στομάχου
ORGAN μύτης
ORGAN βλεννογόνο
PLANT πικραγγουριά
Entities in '  Βότανα  Αρτεμισία, εξαφανίζει το καρκίνο του μασ'
PLANT Αρτεμισία
ORGAN μαστού
ORGAN μαστού
PLANT Αρτεμισία
ORGAN μαστού
PLANT Αρτεμισία
ORGAN μαστού
O

Entities in '  Βότανα  Ασφόδελος: Θεραπευτικές Ιδιότητες, Χρήσε'
ORGAN βλεννογόνου
ORGAN στομάχου
ORGAN ήπατος
ORGAN πνευμονικό
PLANT ασφοδέλου
PLANT ασφοδέλου
PLANT ασφοδέλου
PLANT ασφοδέλου
PLANT αστράγαλος
PLANT ασφοδέλου
PLANT αστράγαλος
PLANT αστράγαλος
ORGAN μαστών
ORGAN όρχεων
PLANT ασφοδέλου
ORGAN μαστών
PLANT ασφοδέλου
PLANT ρίζαις
PLANT πυρός
PLANT σμύρνα
PLANT κρόκο
PLANT κρόκου
PLANT ασφοδέλου
ORGAN δέρματος
PLANT ασφοδέλου
PLANT σμύρνα
ORGAN δοντιού
PLANT ασφοδέλου
PLANT ασφοδέλου
PLANT Χαμίτ
ORGAN μίνας
ORGAN μαλλιών
ORGAN μαλλιά
PLANT ασφοδέλου
PLANT ασφοδέλου
ORGAN δερμάτων
PLANT κριτή
Entities in '  Βότανα  Πώς ξεχωρίζουμε και μαζεύουμε τα βότανα '
PLANT λαχανικά
ORGAN αρτηρίες
PLANT ΑΝΑΓΝΩΡΙΣΗΗ
PLANT χόρτων
Entities in '  Βότανα  Οι Αντικαταθλιπτικές Ιδιότητες του Βαλσα'
PLANT Βαλσαμόχορτου
ORGAN νευρικό
PLANT μοντέρνου
ORGAN εγκέφαλο
ORGAN εγκεφάλου
PLANT βαλσαμόχορτο
ORGAN μυαλό
PLANT Βαλσαμόχορτο
PLANT Βάλσαμο
PLANT Τσιμιτσιφούγκα
PLANT black cohosh
PLANT βαλσαμόχο

Entities in '  Αποτοξίνωσις  Ginkgo Biloba: Προστατεύει εγκέφαλ'
PLANT Ginkgo Biloba
ORGAN εγκέφαλο
ORGAN όρχεις
ORGAN εγκέφαλο
ORGAN όρχεις
PLANT Ginkgo Biloba
ORGAN εγκέφαλο
ORGAN όρχεις
PLANT Ginkgo Biloba
ORGAN εγκέφαλο
ORGAN όρχεις
PLANT Ginkgo Biloba
ORGAN εγκέφαλο
PLANT Ginkgo Biloba
ORGAN εγκέφαλο
ORGAN όρχεις
PLANT Ginkgo biloba
ORGAN εγκέφαλο
PLANT ginkgo biloba
ORGAN νευρικό
PLANT Ginkgo biloba
PLANT Ginkgo biloba
PLANT Ginkgo Biloba
PLANT ginkgo
ORGAN εγκεφαλικών
ORGAN χλωριούχου
ORGAN εγκεφάλου
ORGAN όρχεων
PLANT Ginkgo biloba
ORGAN εγκέφαλο
ORGAN όρχεις
ORGAN εγκεφάλου
PLANT Ginkgo biloba
ORGAN εγκεφαλικών
PLANT Ginkgo biloba
ORGAN εγκέφαλο
ORGAN όρχεις
ORGAN εγκέφαλο
ORGAN όρχεις
PLANT Ginkgo biloba
ORGAN εγκεφάλου
ORGAN όρχεων
ORGAN εγκεφάλου
ORGAN όρχεων
PLANT Ginkgo biloba
PLANT Ginkgo biloba
PLANT Ginkgo biloba
ORGAN εγκέφαλο
ORGAN αρτηριών
PLANT σταφυλιού
ORGAN πνευμόνων
ORGAN αγγείων
ORGAN καρδιάς
PLANT Ginkgo Biloba
ORGAN εγκεφάλου
Entities in '  Βότανα  Εχινάκεια

PLANT Ίταμος
PLANT Ίταμος
PLANT Ίταμος
ORGAN μαστού
ORGAN ωοθηκών
PLANT πεύκα
PLANT πεύκα
PLANT πεύκων
ORGAN ωοθηκών
ORGAN μαστού
PLANT Ίταμος
PLANT πολλαπλός
PLANT ίταμος
PLANT ίταμος
PLANT Ίταμος
PLANT ιτάμου
PLANT Λαβούρνο
PLANT ίταμος
PLANT Άρτεμις
PLANT Άρτεμις
ORGAN καρδιά
ORGAN ωοθηκών
PLANT ίταμος
ORGAN ωοθηκών
ORGAN ήπατος
ORGAN πνεύμονα
PLANT Ίταμος
PLANT γκραβιόλα
PLANT ΓΚΡΑΒΙΌΛΑ
PLANT GRAVIOLA
PLANT κάνναβης
Entities in '  Βότανα  Βότανα και Μπαχαρικά: Πολύτιμη ασπίδα τη'
PLANT Πιπέρι
PLANT πιπέρι
PLANT πιπέρι
PLANT Μαϊντανός
PLANT μαϊντανός
PLANT Μαϊντανός
PLANT Ρίγανη
PLANT ρίγανη
PLANT ρίγανη
PLANT Γαρίφαλο
PLANT γαρίφαλο
PLANT γαρίφαλο
PLANT Κύμινο
PLANT κύμινο
PLANT Κύμινο
PLANT Τζίντζερ
PLANT τζίντζερ
ORGAN καρδιάς
ORGAN αρτηριών
PLANT τζίντζερ
PLANT Βασιλικός
PLANT βασιλικός
ORGAN εντέρου
PLANT Βασιλικός
ORGAN εγκεφάλου
ORGAN εγκεφάλου
PLANT Κόλιανδρος
PLANT κόλιανδρου
PLANT Κόλιανδρος
PLANT Φασκόμηλο
ORGAN εγκεφαλική
PLANT φασκόμηλο
PLANT Φασκόμηλο
PLANT Θυμάρι
ORGA